In [59]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd
pd.set_option('display.float_format', '{:.2f}'.format)


# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestRegressor

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns',100)


In [60]:
df = pd.read_csv("../../datos/02_api_rent_limpieza_cols.csv")
df.sample()

,propertyType,status,price,size,rooms,bathrooms,floor,neighborhood,address,district,province,municipality,exterior,hasLift,hasPlan,has3DTour,has360,distance
290,flat,good,730.00,35.00,1,1,2,Chueca-Justicia,Hernán Cortes,Centro,Madrid,Madrid,False,True,False,False,False,760


# Gestión de Valores erróneos
Vamos a revisarlos en:
- Precio
- pricebyarea
- size
- rooms
- bathrooms


In [61]:
df["province"].unique()

array(['Toledo', 'Madrid', 'Guadalajara', 'Segovia', 'Ávila'],
      dtype=object)

In [62]:
df["price"].unique()

array([550., 750., 400., 450., 590., 684., 600., 700., 695., 650., 675.,
       500., 680., 747., 730., 640., 530., 625., 720., 699., 620., 595.,
       725., 666., 630., 635., 740., 690., 624., 633., 645., 735., 705.,
       710., 610., 580., 560., 670., 723., 660., 692., 609., 718., 694.,
       658., 728., 495., 715., 475., 470., 749., 667., 340., 525., 685.,
       460., 350., 430., 380., 733., 605., 480., 745.])

No se ven anómalos

### size

In [63]:
df["size"].nlargest(5)

0     371.00
407   341.00
260   238.00
71    230.00
237   216.00
Name: size, dtype: float64

In [64]:
df["size"].nsmallest(5)

335   20.00
169   23.00
280   23.00
34    25.00
133   25.00
Name: size, dtype: float64

No parece erróneos

### rooms

In [65]:
df["rooms"].unique()

array([6, 1, 2, 3, 0, 4])

0 es que son estudios

### bathrooms

In [66]:
df["bathrooms"].unique()

array([3, 1, 2])

No se ven valores erróneos

# Corregir "floor"
- Vamos a pasar todo a texto para que sea una categórica

In [67]:
df["floor"].unique()

array([nan, '3', 'bj', '2', '1', '5', 'en', '4', 'st', '8', '7', '6',
       '14', 'ss'], dtype=object)

### Generamos un diccionario para aplicar los cambios

In [68]:
diccionario_pisos = {
    "1": "primero",
    "2": "segundo",
    "3": "tercero",
    "4": "cuarto",
    "5": "quinto",
    "6": "sexto",
    "7": "septimo",
    "8": "octavo",
    "14": "decimo cuarto",
    "bj": "bajo",
    "en": "entreplanta",
    "ss": "sotano",
    "st": "sotano",
}

df["floor"] = df["floor"].map(diccionario_pisos)
df["floor"].unique()


array([nan, 'tercero', 'bajo', 'segundo', 'primero', 'quinto',
       'entreplanta', 'cuarto', 'sotano', 'octavo', 'septimo', 'sexto',
       'decimo cuarto'], dtype=object)

# Corregir Bathrooms
- Vamos a pasar todo a texto para que sea una categórica

In [69]:
df["bathrooms"].unique()

array([3, 1, 2])

In [70]:
df.loc[df["bathrooms"] == 1, "bathrooms"] = "1 aseo"
df.loc[df["bathrooms"] == 2, "bathrooms"] = "2 aseos"
df.loc[df["bathrooms"] == 3, "bathrooms"] = "3 aseos"

In [71]:
df["bathrooms"].unique()

array(['3 aseos', '1 aseo', '2 aseos'], dtype=object)

# Corregir rooms
- Vamos a pasar todo a texto para que sea una categórica

In [72]:
df["rooms"].unique()

array([6, 1, 2, 3, 0, 4])

In [73]:
df.loc[df["rooms"] == 0, "rooms"] = "sin habitaciones"
df.loc[df["rooms"] == 1, "rooms"] = "1 habitacion"
df.loc[df["rooms"] == 2, "rooms"] = "2 habitaciones"
df.loc[df["rooms"] == 3, "rooms"] = "3 habitaciones"
df.loc[df["rooms"] == 4, "rooms"] = "4 habitaciones"
df.loc[df["rooms"] == 6, "rooms"] = "6 habitaciones"

In [74]:
df["rooms"].unique()

array(['6 habitaciones', '1 habitacion', '2 habitaciones',
       '3 habitaciones', 'sin habitaciones', '4 habitaciones'],
      dtype=object)

# Reducción / Simplificación de columnas
- Vamos a quitarnos todo lo que:
    - Especifique demasiado para el precio
- Vamos a simplificar todo lo que:
    - Especifique demasiado, pero al generalizarlo, aporte valor

In [75]:
df.sample()

,propertyType,status,price,size,rooms,bathrooms,floor,neighborhood,address,district,province,municipality,exterior,hasLift,hasPlan,has3DTour,has360,distance
339,flat,good,750.00,87.00,3 habitaciones,1 aseo,bajo,NaN,Calle Ecuador,La Espinilla - Parque Blanco,Madrid,Coslada,True,True,False,False,False,12519


### Dropeamos
- address: No se puede generalizar

In [76]:
df.drop(columns="address",inplace=True)

### Generalizamos
- distance: Lo haremos por rangos de distancia al centro

In [77]:
df["distance"].nlargest()

330    59919
359    59674
361    59601
293    59351
352    59106
Name: distance, dtype: int64

In [78]:
df["distance"].nsmallest()

133    183
154    470
34     533
168    625
394    691
Name: distance, dtype: int64

In [79]:
df["distancia_centro"] = "unknown"
df.loc[df["distance"].between(0,1000,inclusive="left"), "distancia_centro"] = "Menos de 1 km"
df.loc[df["distance"].between(1000,5000,inclusive="left"), "distancia_centro"] = "Entre 1 y 5 km"
df.loc[df["distance"].between(5000,10000,inclusive="left"), "distancia_centro"] =  "Entre 5 y 10 km"
df.loc[df["distance"].between(10000,20000,inclusive="left"), "distancia_centro"] = "Entre 10 y 20 km"
df.loc[df["distance"].between(20000,30000,inclusive="left"), "distancia_centro"] = "Entre 20 y 30 km"
df.loc[df["distance"].between(30000,40000,inclusive="left"), "distancia_centro"] = "Entre 30 y 40 km"
df.loc[df["distance"].between(40000,50000,inclusive="both"), "distancia_centro"] = "Entre 40 y 50 km"
df.loc[df["distance"] > 50000, "distancia_centro"] = "Mas de 50 km"

In [80]:
df["distancia_centro"].value_counts()

distancia_centro
Entre 5 y 10 km     82
Entre 20 y 30 km    74
Entre 10 y 20 km    65
Entre 30 y 40 km    62
Entre 1 y 5 km      62
Entre 40 y 50 km    45
Mas de 50 km        40
Menos de 1 km       12
Name: count, dtype: int64

# Dropeamos distance

In [81]:
df.drop(columns="distance",inplace=True)

# Gestionar nulos variables categóricas
- Las booleanas las volvemos categóricas
- Los nans los pondremos en desconocido

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   propertyType      442 non-null    object 
 1   status            424 non-null    object 
 2   price             442 non-null    float64
 3   size              442 non-null    float64
 4   rooms             442 non-null    object 
 5   bathrooms         442 non-null    object 
 6   floor             361 non-null    object 
 7   neighborhood      174 non-null    object 
 8   district          333 non-null    object 
 9   province          442 non-null    object 
 10  municipality      442 non-null    object 
 11  exterior          442 non-null    bool   
 12  hasLift           411 non-null    object 
 13  hasPlan           424 non-null    object 
 14  has3DTour         424 non-null    object 
 15  has360            424 non-null    object 
 16  distancia_centro  442 non-null    object 
dt

### propertyType

In [83]:
df["propertyType"].isnull().sum()

np.int64(0)

### status

In [84]:
df["status"].isnull().sum()

np.int64(18)

In [85]:
df["status"]= df["status"].fillna("desconocido")
df["status"].unique()

array(['good', 'desconocido', 'newdevelopment', 'renew'], dtype=object)

In [86]:
df["status"].isnull().sum()

np.int64(0)

### floor

In [87]:
df["floor"].isnull().sum()

np.int64(81)

In [88]:
df["floor"]= df["floor"].fillna("desconocido")
df["floor"].unique()

array(['desconocido', 'tercero', 'bajo', 'segundo', 'primero', 'quinto',
       'entreplanta', 'cuarto', 'sotano', 'octavo', 'septimo', 'sexto',
       'decimo cuarto'], dtype=object)

In [89]:
df["floor"].isnull().sum()

np.int64(0)

### neighborhood (Dropear Columna)
- Demasiados nulos, no tiene sentido

In [90]:
df["neighborhood"].isnull().sum()

np.int64(268)

In [91]:
df.drop(columns="neighborhood",inplace=True)

### district
- Tal vez especifique demasiado

In [92]:
df["district"].isnull().sum()

np.int64(109)

In [93]:
df["district"]= df["district"].fillna("desconocido")
df["district"].isnull().sum()

np.int64(0)

### province

In [94]:
df["province"].isnull().sum()

np.int64(0)

### municipality

In [95]:
df["municipality"].isnull().sum()

np.int64(0)

###  exterior
- Cambiamos de booleano a categorica

In [96]:
df["exterior"].isnull().sum()

np.int64(0)

In [97]:
df["exterior"].unique()

array([False,  True])

In [98]:
df.loc[df["exterior"] == True, "exterior"] = "vista exterior"
df.loc[df["exterior"] == False, "exterior"] = "vista interior"
df["exterior"].unique()

array(['vista interior', 'vista exterior'], dtype=object)

### hasLift 
- Cambiamos de booleano a categorica

In [99]:
df["hasLift"].isnull().sum()

np.int64(31)

In [100]:
df["hasLift"].unique()

array([nan, True, False], dtype=object)

In [101]:
df.loc[df["hasLift"] == True, "hasLift"] = "tiene ascensor"
df.loc[df["hasLift"] == False, "hasLift"] = "no tiene ascensor"
df["hasLift"] = df["hasLift"].fillna("desconocido")
df["hasLift"].unique()

array(['desconocido', 'tiene ascensor', 'no tiene ascensor'], dtype=object)

In [102]:
df["hasLift"].isnull().sum()

np.int64(0)

### hasPlan
- Cambiamos de booleano a categorica


In [103]:
df["hasPlan"].isnull().sum()

np.int64(18)

In [104]:
df.loc[df["hasPlan"] == True, "hasPlan"] = "tiene planos"
df.loc[df["hasPlan"] == False, "hasPlan"] = "no tiene planos"
df["hasPlan"] = df["hasPlan"].fillna("desconocido")
df["hasPlan"].unique()

array(['no tiene planos', 'tiene planos', 'desconocido'], dtype=object)

In [105]:
df["hasPlan"].isnull().sum()

np.int64(0)

### has3DTour (Eliminar)
- No tenemos datos que marquen distintos pesos


In [106]:
df.drop(columns="has3DTour", inplace=True)

### has360
- Cambiamos de booleano a categorica

In [107]:
df["has360"].isnull().sum()

np.int64(18)

In [108]:
c = df["has360"].isnull() == True
df[c]


,propertyType,status,price,size,rooms,bathrooms,floor,district,province,municipality,exterior,hasLift,hasPlan,has360,distancia_centro
7,studio,desconocido,684.00,45.00,sin habitaciones,1 aseo,desconocido,desconocido,Madrid,Madrid,vista exterior,desconocido,desconocido,NaN,Entre 10 y 20 km
37,flat,desconocido,750.00,95.00,3 habitaciones,2 aseos,desconocido,desconocido,Toledo,Illescas,vista exterior,desconocido,desconocido,NaN,Entre 30 y 40 km
66,flat,desconocido,700.00,50.00,1 habitacion,2 aseos,desconocido,desconocido,Madrid,Camarma de Esteruelas,vista exterior,desconocido,desconocido,NaN,Entre 30 y 40 km
67,duplex,desconocido,750.00,35.00,1 habitacion,1 aseo,desconocido,desconocido,Madrid,Madrid,vista exterior,desconocido,desconocido,NaN,Entre 1 y 5 km
131,flat,desconocido,600.00,55.00,1 habitacion,1 aseo,desconocido,desconocido,Madrid,Daganzo de Arriba,vista exterior,desconocido,desconocido,NaN,Entre 20 y 30 km
137,flat,desconocido,550.00,106.00,2 habitaciones,1 aseo,desconocido,desconocido,Toledo,Ocaña,vista exterior,desconocido,desconocido,NaN,Mas de 50 km
149,flat,desconocido,630.00,54.00,1 habitacion,2 aseos,desconocido,desconocido,Madrid,El Álamo,vista exterior,desconocido,desconocido,NaN,Entre 30 y 40 km
169,studio,desconocido,700.00,23.00,sin habitaciones,1 aseo,desconocido,desconocido,Madrid,Madrid,vista exterior,desconocido,desconocido,NaN,Entre 1 y 5 km
179,duplex,desconocido,600.00,80.00,1 habitacion,1 aseo,desconocido,desconocido,Madrid,Madrid,vista interior,desconocido,desconocido,NaN,Entre 5 y 10 km
180,duplex,desconocido,600.00,80.00,1 habitacion,1 aseo,desconocido,desconocido,Madrid,Madrid,vista interior,desconocido,desconocido,NaN,Entre 5 y 10 km


#### Habrá que ver si estos datos merece la pena tenerlos, hay muchas cosas desconocidas

In [109]:
df.loc[df["has360"] == True, "has360"] = "tiene fotos 360"
df.loc[df["has360"] == False, "has360"] = "no tiene fotos 360"
df["has360"] = df["has360"].fillna("desconocido")
df["has360"].unique()

array(['no tiene fotos 360', 'tiene fotos 360', 'desconocido'],
      dtype=object)

In [110]:
df["has360"].isnull().sum()

np.int64(0)

### distancia_centro

In [111]:
df["distancia_centro"].isnull().sum()

np.int64(0)

# Gestión nulos numéricas
- Sin tocar la VR (Precio).
- Rellenaremos con IterativeImputer y RandomForest
    - Las relaciones son complejas

In [112]:
df.isnull().sum()

propertyType        0
status              0
price               0
size                0
rooms               0
bathrooms           0
floor               0
district            0
province            0
municipality        0
exterior            0
hasLift             0
hasPlan             0
has360              0
distancia_centro    0
dtype: int64

# Eliminar district
- Tiene demasiados "Desconocido"
- Tiene demasiados datos que especifican demasiado

In [113]:
df.drop(columns="district", inplace=True)

# Eliminar hasPlan, has360 y Province

In [114]:
cols_chao = ["hasPlan","has360","province"]
df.drop(columns=cols_chao,inplace=True)

# Guardamos

In [115]:
df.to_csv("../../datos/03_api_rent_sin_nulos.csv",index=False)